# Topic Modeling using LDA

In [1]:
# # To be run only once
# if 0 == 1:
#     !pip install gensim
#     !pip install PyLDAvis
#     !pip install spacy
#     !python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt

import re
import spacy
import tqdm

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

from nltk.corpus import stopwords
import nltk
import en_core_web_sm

from pprint import pprint

import pyLDAvis.gensim
import pickle 
import pyLDAvis

import time
from collections import Counter
import random

In [73]:
random.seed(3)

KeyboardInterrupt: 

In [74]:
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', None)

In [4]:
# nltk.download('stopwords')

## Preprocessing data

In [5]:
start = time.time()

In [6]:
# one_drive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"
# one_drive_path = "C:/Users/asaid/The Estée Lauder Companies Inc/TeamAnis - General/"

In [7]:
# reviews = pd.read_pickle('reviews.pickle')

# cols = ['type', 'onlinepost_id', 'source_product_identifier', 'onlinestatement_id',
#        'date', 'title', 'description', 'geography', 'channel', 'product_id',
#        'rating', 'sentiment']
# reviews = reviews[cols]

# reviews = reviews[reviews['geography']=='USA']
# reviews.drop(columns=['geography'], inplace=True)

# reviews.to_pickle('reviews_filtered.pickle')

In [8]:
# reviews = pd.read_pickle('reviews_filtered.pickle')

In [9]:
# # Creating date columns in the right dtype and dropping the day of the month: 2019-02-24 => 2019-02-01
# reviews.loc[:,'clean_date'] = pd.to_datetime(reviews['date'], errors='coerce')
# if reviews['clean_date'].isna().sum() > 0:
#     print('{} rows have been dropped because the date format is wrong.'.format(reviews['clean_date'].isna().sum()))
#     display(reviews.loc[reviews['clean_date'].isna(), 'date'])
#     reviews = reviews.dropna(subset='date')
# reviews['date'] = reviews['clean_date']  
# reviews = reviews.drop('clean_date', axis=1)
# reviews['date'] = reviews['date'].dt.to_period('m')

# # Checking for missing data (NA => -1)
# if reviews['rating'].isna().sum()>0:
#     print('{} rows are missing ratings'.format(reviews['rating'].isna().sum()))
#     reviews.loc[:,'rating'] = reviews['rating'].fillna(-1).astype(int)

# if reviews['sentiment'].isna().sum()>0:
#     print('{} rows are missing sentiments'.format(reviews['sentiment'].isna().sum()))
#     reviews.loc[:,'sentiment'] = reviews['sentiment'].fillna(-1).astype(int)
    
# # Transforming rating and sentiment to dummy variables (one-hot encoding)
# reviews.loc[:,'sentiment'] = reviews['sentiment'].str.lower()
# reviews.loc[:,'rating'] = reviews['rating'].astype(int)
# reviews = pd.concat([reviews, pd.get_dummies(data=reviews[['rating','sentiment']], columns=['rating','sentiment'], dtype=int)], axis=1)

# # Readding NAs data to ratings
# reviews.loc[reviews['rating']==-1,'rating'] = np.nan
# reviews.loc[reviews['sentiment']==-1,'sentiment'] = np.nan

# # Transforming sentiment to integer data (positive:1; netural:0, negative:-1)
# reviews.loc[:,'sentiment'] = reviews['sentiment_positive'] - reviews['sentiment_negative']

# # Aggregating RR data by OnlinePost_ID

# # Creating a column to count the number of statements by review once aggreagtion happens
# reviews['nb_statements'] = reviews['sentiment']

# reviews = reviews.groupby(['type',
#                            'channel',
#                            'source_product_identifier',  
#                            'date',
#                            'onlinepost_id']).agg({
#     'description': lambda x:'. '.join(list(x)),
#     'nb_statements':'count',
#     'rating':'first',
#     'rating_1':'first',
#     'rating_2':'first',
#     'rating_3':'first',
#     'rating_4':'first',
#     'rating_5':'first',
#     'sentiment_negative':'sum',
#     'sentiment_neutral':'sum',
#     'sentiment_positive':'sum',
#     'sentiment':'mean'
# }).reset_index()

# # Normalize the one hot sentiment encoding counts (sentiment_negative, sentiment_neutral, sentiment_positive) by the nb_statement.
# reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']] = reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']].div(reviews['nb_statements'], axis=0)

# reviews.to_pickle('reviews_wrangled.pickle')

In [ ]:
# reviews = pd.read_pickle('reviews_wrangled.pickle')

In [88]:
# Adding product related words to the stop words
stop_words = stopwords.words('english')
stop_words.extend(['from'])

brands = pd.read_csv('elc_brands.csv', encoding='ISO-8859-1')
catalogue = pd.read_csv('elc_catalogue.csv', encoding='ISO-8859-1')

stop_words.extend(list(set(str(w).lower() for w in ' '.join(brands['ELC_Brand'].unique().tolist() +
                                          catalogue['Major_Category'].unique().tolist() +
                                          catalogue['Application'].unique().tolist() +
                                          catalogue['Category'].unique().tolist() +
                                          catalogue['SubCategory'].unique().tolist() 
                                         ).replace('/', ' ').split())))
# Cache stop_words into hash
stop_words = Counter(stop_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def preprocess(sentences, stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    for sentence in sentences:
        doc = nlp(' '.join([token for token in gensim.utils.simple_preprocess(str(sentence), deacc=True) if token not in stop_words]) )
        yield([token.lemma_ for token in doc if token.pos_ in allowed_postags])

In [93]:
# reviews['tokens'] = list(tqdm.tqdm(preprocess(reviews['description'].values.tolist(), stop_words), position=0, leave=True, total=len(reviews)))

100%|█████████████████████████████████████████████████████████████████████| 4636608/4636608 [3:20:10<00:00, 386.04it/s]


In [94]:
# reviews.to_pickle('reviews_wrangled_w_tokens.pickle')

In [99]:
reviews

,type,channel,source_product_identifier,date,onlinepost_id,description,nb_statements,rating,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment,tokens
0,Cosmetics,Amazon USA,1456987763,2017-03,OnlinePost_20191120_142698357,"It was not what I was expecting, not really go...",4,2.0,0,1,0,0,0,0.250000,0.250000,0.500000,0.250000,"[expect, really, good, quality, wait, week, ma..."
1,Cosmetics,Amazon USA,1837994757,2015-02,OnlinePost_20191120_140661460,It leaves your face feeling silky.,1,5.0,0,0,0,0,1,0.000000,0.000000,1.000000,1.000000,"[leave, feel, silky]"
2,Cosmetics,Amazon USA,1837994757,2015-02,OnlinePost_20200301_d0f7f698-e4d6-412f-8364-d8...,love this serum!. I use this daily after I was...,2,5.0,0,0,0,0,1,0.000000,0.000000,1.000000,1.000000,"[daily, wash, regular]"
3,Cosmetics,Amazon USA,1837994757,2015-03,OnlinePost_20200301_c40a460e-71d5-4fab-becb-1b...,Doesn't make my face oily.. Feels so silky smo...,3,5.0,0,0,0,0,1,0.000000,0.000000,1.000000,1.000000,"[make, oily, feel, silky, smooth, star]"
4,Cosmetics,Amazon USA,1837994757,2015-05,OnlinePost_20191120_141881069,Love this product!,1,5.0,0,0,0,0,1,0.000000,0.000000,1.000000,1.000000,"[love, product]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4636603,Skincare,Ulta,xlsImpprod930218,2019-01,OnlinePost_20191027_184578422,thickens my lashes just the right amount and k...,1,3.0,0,0,1,0,0,0.000000,0.000000,1.000000,1.000000,"[lash, right, amount, keep, curl, still, smudg..."
4636604,Skincare,Ulta,xlsImpprod930218,2019-02,OnlinePost_20191027_184466715,This mascara smudges and flakes and the second...,3,2.0,0,1,0,0,0,0.333333,0.000000,0.666667,0.333333,"[smudge, flake, second, put, want, take, feel,..."
4636605,Skincare,Ulta,xlsImpprod930218,2019-02,OnlinePost_20191027_184507492,a little difficult to take off but nothing bad...,4,5.0,0,0,0,0,1,0.500000,0.000000,0.500000,0.000000,"[little, difficult, take, bad, best, use, long..."
4636606,Skincare,Ulta,xlsImpprod930218,2019-02,OnlinePost_20191027_184648927,It's lightweight and feels comfortable on the ...,2,5.0,0,0,0,0,1,0.000000,0.000000,1.000000,1.000000,"[feel, comfortable, lash, good]"


In [ ]:
reviews = reviews.read_pickle('reviews_wrangled_w_tokens.pickle')

## Using class

In [155]:
class TopicModeller():
    def __init__(self, df, column):
        self.df = df.reset_index(drop=True)
        self.column = column
        self.docs = df[column].values.tolist()
    
    def compute_coherence_values(self, corpus, id2word, text, k, a, b):

        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=k, 
                                               random_state=3,
                                               chunksize=100,
                                               passes=10,
                                               alpha=a,
                                               eta=b)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')

        return coherence_model_lda.get_coherence()
    
    def train_valid_lda(self,
                        nb_samples: int = 100000,
                        topics_range: list = [3,9],
                        alpha: list = [0.1, 'symmetric','asymmetric'],
                        beta: list = [0.1, 'symmetric']):
        
        nb_samples = min(nb_samples, len(self.docs))
        
        # Training set
        train = random.sample(self.docs, nb_samples)
        
        # Create Dictionary
        self.id2word = corpora.Dictionary(train)
        
        # Term Document Frequency
        self.corpus = [self.id2word.doc2bow(text) for text in train]
        
        # Result dictionary
        self.model_results = {'Topics': [],
                         'Alpha': [],
                         'Beta': [],
                         'Coherence': []
                        }

        # Can take a long time to run
        print('Validating hyperparameters...')
        if 1 == 1:
            pbar = tqdm.tqdm(total=(len(beta)*(len(alpha)+1)*len(topics_range)), position=0, leave=True)
            # iterate through number of topics
            for k in topics_range:
                extended_alpha = alpha+[50/k]
                # iterate through alpha values
                for a in extended_alpha:
                    # iterare through beta values
                    for b in beta:
                        # get the coherence score for the given parameters
                        cv = self.compute_coherence_values(corpus=self.corpus, id2word=self.id2word, text=train,
                                                      k=k, a=a, b=b)
                        # Save the model results
                        self.model_results['Topics'].append(k)
                        self.model_results['Alpha'].append(a)
                        self.model_results['Beta'].append(b)
                        self.model_results['Coherence'].append(cv)
                        print('nb topics: {}, alpha: {}, beta: {}, coherence: {}'.format(k,a,b,cv))
                        pbar.update(1)
        self.cv_results = pd.DataFrame(self.model_results).sort_values("Coherence", ascending=False)
        self.cv_results.to_csv('lda_tuning_results.csv', index=False)
        pbar.close()
        
        self.best_param = self.cv_results.iloc[0]
        self.best_num_topics = int(self.best_param['Topics'])
        self.best_alpha = self.best_param['Alpha']
        self.best_beta = self.best_param['Beta']
        
        # Build LDA model
        print('Training best model...')
        self.lda_model = gensim.models.LdaMulticore(corpus=self.corpus,
                                                   id2word=self.id2word,
                                                   num_topics=self.best_num_topics,
                                                   alpha = self.best_alpha,
                                                   eta = self.best_beta,
                                                   random_state=3,
                                                   chunksize=100,
                                                   passes=10,
                                                   per_word_topics=True)
        print('Done.')
        
    def get_docs_topics(self):
        # Term Document Frequency
        corpus = [self.id2word.doc2bow(text) for text in self.docs]
        self.output = pd.concat([self.df, pd.DataFrame(gensim.matutils.corpus2csc(self.lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,self.best_num_topics+1)])], axis=1)
        return self.output

    def view(self):
        pyLDAvis.enable_notebook()
        LDAvis_prepared = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
        display(LDAvis_prepared)

In [142]:
good_reviews = reviews[reviews['rating']>=4]
neutral_reviews = reviews[(reviews['rating']>2) & (reviews['rating']<4)]
bad_reviews = reviews[reviews['rating']<=2]

In [145]:
print(len(good_reviews))
good_lda = TopicModeller(good_reviews, 'tokens')
good_lda.train_valid_lda()

3605467


  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

Validating hyperparameters...


30it [4:37:23, 554.78s/it]                                                                                             


Training best model...
Done.


In [146]:
display(good_lda.cv_results)

,Topics,Alpha,Beta,Coherence
29,9,5.55556,symmetric,0.529431
28,9,5.55556,0.1,0.528639
23,9,asymmetric,symmetric,0.514659
12,6,asymmetric,0.1,0.514239
13,6,asymmetric,symmetric,0.506266
21,9,symmetric,symmetric,0.506242
22,9,asymmetric,0.1,0.504448
17,6,8.33333,symmetric,0.502483
19,9,0.1,symmetric,0.500803
16,6,8.33333,0.1,0.497481


In [147]:
good_lda.view()

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.185515  0.050049       1        1  11.504065
1      0.304863  0.076978       2        1  11.385640
7     -0.238536  0.326510       3        1  11.344358
4      0.079629  0.008155       4        1  11.161737
3     -0.097432  0.012390       5        1  11.056719
5      0.002551 -0.116474       6        1  11.037022
2      0.047078 -0.006545       7        1  10.965193
0     -0.104379 -0.222222       8        1  10.813502
8     -0.179289 -0.128841       9        1  10.731767, topic_info=               Term          Freq         Total Category  logprob  loglift
24             love  88790.000000  88790.000000  Default  30.0000  30.0000
23             look  51784.000000  51784.000000  Default  29.0000  29.0000
66              use  79123.000000  79123.000000  Default  28.0000  28.0000
94             make  34940.000000  34940.000000  Default  27.0000  27.0000
110            good  45432.000000  45432.000000  Default  26.0000  26.0000
44              try  26186.000000  26186.000000  Default  25.0000  25.0000
50            great  61948.000000  61948.000000  Default  24.0000  24.0000
18               go  35680.000000  35680.000000  Default  23.0000  23.0000
120            easy  18865.000000  18865.000000  Default  22.0000  22.0000
55           little  17893.000000  17893.000000  Default  21.0000  21.0000
395        coverage  16709.000000  16709.000000  Default  20.0000  20.0000
20             last  23678.000000  23678.000000  Default  19.0000  19.0000
132           shade  13907.000000  13907.000000  Default  18.0000  18.0000
42             time  17564.000000  17564.000000  Default  17.0000  17.0000
4              nice  21673.000000  21673.000000  Default  16.0000  16.0000
6             smell  13297.000000  13297.000000  Default  15.0000  15.0000
108             day  29804.000000  29804.000000  Default  14.0000  14.0000
138            lash  12834.000000  12834.000000  Default  13.0000  13.0000
3           natural  13358.000000  13358.000000  Default  12.0000  12.0000
31          perfect  21034.000000  21034.000000  Default  11.0000  11.0000
1              feel  30740.000000  30740.000000  Default  10.0000  10.0000
112            stay  14281.000000  14281.000000  Default   9.0000   9.0000
8              well  33300.000000  33300.000000  Default   8.0000   8.0000
104            full  10811.000000  10811.000000  Default   7.0000   7.0000
205       beautiful  10593.000000  10593.000000  Default   6.0000   6.0000
59          product  73749.000000  73749.000000  Default   5.0000   5.0000
16            color  10996.000000  10996.000000  Default   4.0000   4.0000
22             long  24423.000000  24423.000000  Default   3.0000   3.0000
381            oily  10528.000000  10528.000000  Default   2.0000   2.0000
280           super   9240.000000   9240.000000  Default   1.0000   1.0000
127            brow   6466.593262   6467.497559   Topic1  -3.9596   2.1623
907         eyebrow   2419.318604   2420.221436   Topic1  -4.9428   2.1621
198           brush   5595.109375   5597.710449   Topic1  -4.1044   2.1620
1272          shape   1698.358765   1699.261963   Topic1  -5.2966   2.1619
330            fill   1407.639038   1408.541748   Topic1  -5.4844   2.1618
509            lose    498.418243    499.321228   Topic1  -6.5226   2.1607
2579         figure    470.789673    471.693024   Topic1  -6.5796   2.1606
608            grow    460.961426    461.864075   Topic1  -6.6007   2.1605
283           write    458.559326    459.462067   Topic1  -6.6060   2.1605
1468           draw    596.722046    597.982605   Topic1  -6.3426   2.1604
1582          waste    422.471924    423.374542   Topic1  -6.6879   2.1603
868            hair    404.729248    405.631866   Topic1  -6.7308   2.1602
1672         stroke    400.240021    401.142700   Topic1  -6.7420   2.1602
1225         sparse    366.554352    367.456970   Topic1  -6.82

In [156]:
print(len(bad_reviews))
bad_lda = TopicModeller(bad_reviews, 'tokens')
bad_lda.train_valid_lda()
bad_lda.view()

585520


  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

Validating hyperparameters...


100%|███████████████████████████████████████████████████████████████████████████████| 16/16 [1:07:46<00:00, 254.17s/it]


Training best model...
Done.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.006124  0.027560       1        1  18.308479
8      0.129373 -0.044485       2        1  16.898638
2      0.106756  0.000642       3        1  12.217730
5      0.169401  0.006427       4        1  10.379268
4      0.029508 -0.104814       5        1   9.579919
7     -0.092060 -0.284646       6        1   9.128814
3      0.150007  0.200029       7        1   8.353272
0     -0.219805  0.103063       8        1   7.695968
6     -0.267054  0.096224       9        1   7.437911, topic_info=               Term          Freq         Total Category  logprob  loglift
58             feel  13697.000000  13697.000000  Default  30.0000  30.0000
194           smell   9083.000000   9083.000000  Default  29.0000  29.0000
94          product  44124.000000  44124.000000  Default  28.0000  28.0000
43             look  24319.000000  24319.000000  Default  27.0000  27.0000
175            work  13544.000000  13544.000000  Default  26.0000  26.0000
211            star   6378.000000   6378.000000  Default  25.0000  25.0000
193           shade   6122.000000   6122.000000  Default  24.0000  24.0000
22             lash   5967.000000   5967.000000  Default  23.0000  23.0000
17              use  35580.000000  35580.000000  Default  22.0000  22.0000
61             make  19080.000000  19080.000000  Default  21.0000  21.0000
63             oily   6924.000000   6924.000000  Default  20.0000  20.0000
137           break   7063.000000   7063.000000  Default  19.0000  19.0000
55           bottle   4065.000000   4065.000000  Default  18.0000  18.0000
19             year   3841.000000   3841.000000  Default  17.0000  17.0000
73             last   6024.000000   6024.000000  Default  16.0000  16.0000
324             old   3752.000000   3752.000000  Default  15.0000  15.0000
54              bad   9586.000000   9586.000000  Default  14.0000  14.0000
535          sample   5109.000000   5109.000000  Default  13.0000  13.0000
339         receive   5518.000000   5518.000000  Default  12.0000  12.0000
8             money   6413.000000   6413.000000  Default  11.0000  11.0000
306        coverage   5009.000000   5009.000000  Default  10.0000  10.0000
147            dark   4342.000000   4342.000000  Default   9.0000   9.0000
507       sensitive   3536.000000   3536.000000  Default   8.0000   8.0000
295           order   3463.000000   3463.000000  Default   7.0000   7.0000
38             come   6960.000000   6960.000000  Default   6.0000   6.0000
354         formula   4943.000000   4943.000000  Default   5.0000   5.0000
84              try  18349.000000  18349.000000  Default   4.0000   4.0000
119            open   3077.000000   3077.000000  Default   3.0000   3.0000
209          sticky   3457.000000   3457.000000  Default   2.0000   2.0000
105          change   3007.000000   3007.000000  Default   1.0000   1.0000
257            brow   1568.154907   1568.976318   Topic1  -5.1921   1.6973
161            pump   1382.272705   1383.094238   Topic1  -5.3183   1.6972
436            hear   1041.041260   1041.868042   Topic1  -5.6018   1.6970
143           other    968.812744    969.634277   Topic1  -5.6737   1.6970
1011            low    692.621948    693.443481   Topic1  -6.0093   1.6966
509          absorb    571.474426    572.295837   Topic1  -6.2016   1.6964
2069          write    522.500793    523.322205   Topic1  -6.2912   1.6962
511     expectation    424.113678    424.935059   Topic1  -6.4998   1.6959
2279            odd    400.522308    401.343750   Topic1  -6.5570   1.6958
1637        include    390.474579    391.296051   Topic1  -6.5824   1.6957
1103           face    387.609955    388.431335   Topic1  -6.5898   1.6957
92          lancome    366.736664    367.558105   Topic1  -6.6451   1.6956
874         visible    366.030914    366.852356   Topic1  -6.6471   1.6956
1181         market    410.493652    411.416962   Topic1  -6.53

In [157]:
display(bad_lda.cv_results)

,Topics,Alpha,Beta,Coherence
10,9,symmetric,0.1,0.543806
13,9,asymmetric,symmetric,0.539759
9,9,0.1,symmetric,0.538955
12,9,asymmetric,0.1,0.538672
15,9,5.55556,symmetric,0.534416
14,9,5.55556,0.1,0.531963
11,9,symmetric,symmetric,0.531838
8,9,0.1,0.1,0.524311
4,3,asymmetric,0.1,0.479731
2,3,symmetric,0.1,0.479557


In [158]:
bad_lda.view()

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.006124  0.027560       1        1  18.307310
8      0.129373 -0.044485       2        1  16.898687
2      0.106756  0.000642       3        1  12.223215
5      0.169401  0.006427       4        1  10.378328
4      0.029508 -0.104814       5        1   9.576618
7     -0.092060 -0.284646       6        1   9.134280
3      0.150007  0.200029       7        1   8.350942
0     -0.219805  0.103063       8        1   7.692784
6     -0.267054  0.096224       9        1   7.437835, topic_info=               Term          Freq         Total Category  logprob  loglift
58             feel  13696.000000  13696.000000  Default  30.0000  30.0000
194           smell   9088.000000   9088.000000  Default  29.0000  29.0000
94          product  44120.000000  44120.000000  Default  28.0000  28.0000
43             look  24317.000000  24317.000000  Default  27.0000  27.0000
175            work  13544.000000  13544.000000  Default  26.0000  26.0000
211            star   6381.000000   6381.000000  Default  25.0000  25.0000
193           shade   6120.000000   6120.000000  Default  24.0000  24.0000
22             lash   5965.000000   5965.000000  Default  23.0000  23.0000
17              use  35581.000000  35581.000000  Default  22.0000  22.0000
61             make  19081.000000  19081.000000  Default  21.0000  21.0000
63             oily   6924.000000   6924.000000  Default  20.0000  20.0000
137           break   7065.000000   7065.000000  Default  19.0000  19.0000
55           bottle   4063.000000   4063.000000  Default  18.0000  18.0000
19             year   3841.000000   3841.000000  Default  17.0000  17.0000
73             last   6026.000000   6026.000000  Default  16.0000  16.0000
324             old   3752.000000   3752.000000  Default  15.0000  15.0000
54              bad   9589.000000   9589.000000  Default  14.0000  14.0000
535          sample   5107.000000   5107.000000  Default  13.0000  13.0000
339         receive   5516.000000   5516.000000  Default  12.0000  12.0000
8             money   6415.000000   6415.000000  Default  11.0000  11.0000
306        coverage   5009.000000   5009.000000  Default  10.0000  10.0000
147            dark   4341.000000   4341.000000  Default   9.0000   9.0000
507       sensitive   3538.000000   3538.000000  Default   8.0000   8.0000
295           order   3462.000000   3462.000000  Default   7.0000   7.0000
38             come   6959.000000   6959.000000  Default   6.0000   6.0000
354         formula   4943.000000   4943.000000  Default   5.0000   5.0000
84              try  18348.000000  18348.000000  Default   4.0000   4.0000
119            open   3076.000000   3076.000000  Default   3.0000   3.0000
209          sticky   3457.000000   3457.000000  Default   2.0000   2.0000
105          change   3007.000000   3007.000000  Default   1.0000   1.0000
257            brow   1568.053223   1568.874634   Topic1  -5.1921   1.6973
161            pump   1382.183105   1383.004395   Topic1  -5.3183   1.6973
436            hear   1040.973755   1041.800415   Topic1  -5.6018   1.6971
143           other    968.749939    969.571411   Topic1  -5.6737   1.6970
1011            low    692.577026    693.398438   Topic1  -6.0093   1.6967
509          absorb    571.437378    572.258850   Topic1  -6.2016   1.6964
2069          write    522.466919    523.288330   Topic1  -6.2912   1.6963
511     expectation    424.086182    424.907593   Topic1  -6.4998   1.6959
2279            odd    400.496338    401.317810   Topic1  -6.5570   1.6958
1637        include    390.449280    391.270782   Topic1  -6.5824   1.6958
1103           face    387.584839    388.406250   Topic1  -6.5898   1.6958
92          lancome    366.712891    367.534332   Topic1  -6.6451   1.6956
874         visible    366.007172    366.828583   Topic1  -6.6471   1.6956
1181         market    410.467041    411.390350   Topic1  -6.53

In [159]:
print(len(neutral_reviews))
neutral_lda = TopicModeller(neutral_reviews, 'tokens')
neutral_lda.train_valid_lda()
neutral_lda.view()

445621


  0%|                                                                                           | 0/16 [00:00<?, ?it/s]

Validating hyperparameters...


100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [56:47<00:00, 212.99s/it]


Training best model...
Done.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.062524  0.067366       1        1  21.879694
1     -0.071944 -0.062903       2        1  14.926169
2     -0.104341  0.071810       3        1  14.338291
5     -0.018550 -0.045408       4        1  13.470085
6      0.022263  0.077690       5        1   9.733492
3     -0.091289  0.092859       6        1   7.618577
4     -0.217418 -0.198273       7        1   6.491327
7      0.201229  0.204804       8        1   6.294791
8      0.342574 -0.207945       9        1   5.247564, topic_info=               Term          Freq         Total Category  logprob  loglift
12             last   9108.000000   9108.000000  Default  30.0000  30.0000
82             long   9844.000000   9844.000000  Default  29.0000  29.0000
38            apply  11579.000000  11579.000000  Default  28.0000  28.0000
192        coverage   8362.000000   8362.000000  Default  27.0000  27.0000
64              use  37399.000000  37399.000000  Default  26.0000  26.0000
50             look  24260.000000  24260.000000  Default  25.0000  25.0000
62            smell   8271.000000   8271.000000  Default  24.0000  24.0000
77             star   5488.000000   5488.000000  Default  23.0000  23.0000
120            lash   6290.000000   6290.000000  Default  22.0000  22.0000
56               go  15428.000000  15428.000000  Default  21.0000  21.0000
34             oily   6677.000000   6677.000000  Default  20.0000  20.0000
4               day   9649.000000   9649.000000  Default  19.0000  19.0000
119           shade  10062.000000  10062.000000  Default  18.0000  18.0000
9              good  23361.000000  23361.000000  Default  17.0000  17.0000
46             feel  18808.000000  18808.000000  Default  16.0000  16.0000
20          product  42062.000000  42062.000000  Default  15.0000  15.0000
115          sample   8329.000000   8329.000000  Default  14.0000  14.0000
138          review   4723.000000   4723.000000  Default  13.0000  13.0000
94             dark   6298.000000   6298.000000  Default  12.0000  12.0000
102             see   6552.000000   6552.000000  Default  11.0000  11.0000
40             easy   4804.000000   4804.000000  Default  10.0000  10.0000
235            stay   5994.000000   5994.000000  Default   9.0000   9.0000
325           price   5154.000000   5154.000000  Default   8.0000   8.0000
125            wear   9273.000000   9273.000000  Default   7.0000   7.0000
44              dry   4125.000000   4125.000000  Default   6.0000   6.0000
39            color   6255.000000   6255.000000  Default   5.0000   5.0000
52             nice  15426.000000  15426.000000  Default   4.0000   4.0000
87          receive   5990.000000   5990.000000  Default   3.0000   3.0000
501          smooth   4718.000000   4718.000000  Default   2.0000   2.0000
127      difference   3178.000000   3178.000000  Default   1.0000   1.0000
39            color   6254.856445   6255.663086   Topic1  -4.0152   1.5195
616        lipstick   4169.349121   4170.155762   Topic1  -4.4208   1.5194
786         pigment   3450.402832   3451.209961   Topic1  -4.6101   1.5194
169            pink   3440.941162   3441.748291   Topic1  -4.6129   1.5194
117          shadow   3080.158936   3080.966064   Topic1  -4.7236   1.5193
348          bright   2362.371338   2363.178467   Topic1  -4.9889   1.5193
372           stain   1368.467896   1369.274780   Topic1  -5.5349   1.5190
309       highlight   1360.572510   1361.379395   Topic1  -5.5407   1.5190
366           sheer   1314.925659   1315.732544   Topic1  -5.5748   1.5190
614          colour   1259.774658   1260.581543   Topic1  -5.6177   1.5190
675          subtle   1135.400635   1136.207520   Topic1  -5.7216   1.5189
302         neutral   1034.503174   1035.310059   Topic1  -5.8147   1.5188
694      complexion    920.069031    920.876160   Topic1  -5.9319   1.5187
950            nude    789.255615    790.062622   Topic1  -6.08

In [160]:
display(neutral_lda.cv_results)

,Topics,Alpha,Beta,Coherence
12,9,asymmetric,0.1,0.494624
13,9,asymmetric,symmetric,0.494306
11,9,symmetric,symmetric,0.493490
8,9,0.1,0.1,0.492105
10,9,symmetric,0.1,0.489150
14,9,5.55556,0.1,0.488979
15,9,5.55556,symmetric,0.484942
9,9,0.1,symmetric,0.483010
5,3,asymmetric,symmetric,0.423331
4,3,asymmetric,0.1,0.421154


In [161]:
neutral_lda.view()

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.062524  0.067366       1        1  21.891186
1     -0.071944 -0.062903       2        1  14.926285
2     -0.104341  0.071810       3        1  14.331020
5     -0.018550 -0.045408       4        1  13.472164
6      0.022263  0.077690       5        1   9.733612
3     -0.091289  0.092859       6        1   7.614524
4     -0.217418 -0.198273       7        1   6.491765
7      0.201229  0.204804       8        1   6.294011
8      0.342574 -0.207945       9        1   5.245436, topic_info=               Term          Freq         Total Category  logprob  loglift
12             last   9104.000000   9104.000000  Default  30.0000  30.0000
82             long   9840.000000   9840.000000  Default  29.0000  29.0000
38            apply  11578.000000  11578.000000  Default  28.0000  28.0000
192        coverage   8357.000000   8357.000000  Default  27.0000  27.0000
64              use  37401.000000  37401.000000  Default  26.0000  26.0000
50             look  24268.000000  24268.000000  Default  25.0000  25.0000
62            smell   8267.000000   8267.000000  Default  24.0000  24.0000
77             star   5488.000000   5488.000000  Default  23.0000  23.0000
120            lash   6290.000000   6290.000000  Default  22.0000  22.0000
56               go  15429.000000  15429.000000  Default  21.0000  21.0000
34             oily   6678.000000   6678.000000  Default  20.0000  20.0000
4               day   9648.000000   9648.000000  Default  19.0000  19.0000
119           shade  10065.000000  10065.000000  Default  18.0000  18.0000
9              good  23357.000000  23357.000000  Default  17.0000  17.0000
46             feel  18803.000000  18803.000000  Default  16.0000  16.0000
20          product  42060.000000  42060.000000  Default  15.0000  15.0000
115          sample   8330.000000   8330.000000  Default  14.0000  14.0000
138          review   4722.000000   4722.000000  Default  13.0000  13.0000
94             dark   6298.000000   6298.000000  Default  12.0000  12.0000
102             see   6553.000000   6553.000000  Default  11.0000  11.0000
40             easy   4804.000000   4804.000000  Default  10.0000  10.0000
235            stay   5993.000000   5993.000000  Default   9.0000   9.0000
325           price   5152.000000   5152.000000  Default   8.0000   8.0000
125            wear   9274.000000   9274.000000  Default   7.0000   7.0000
44              dry   4125.000000   4125.000000  Default   6.0000   6.0000
39            color   6258.000000   6258.000000  Default   5.0000   5.0000
52             nice  15428.000000  15428.000000  Default   4.0000   4.0000
87          receive   5991.000000   5991.000000  Default   3.0000   3.0000
501          smooth   4718.000000   4718.000000  Default   2.0000   2.0000
127      difference   3178.000000   3178.000000  Default   1.0000   1.0000
39            color   6258.137695   6258.944336   Topic1  -4.0152   1.5190
616        lipstick   4171.536133   4172.342773   Topic1  -4.4208   1.5189
786         pigment   3452.213135   3453.020264   Topic1  -4.6101   1.5189
169            pink   3442.746338   3443.553223   Topic1  -4.6129   1.5189
117          shadow   3081.774902   3082.582031   Topic1  -4.7236   1.5188
348          bright   2363.610596   2364.417480   Topic1  -4.9889   1.5187
372           stain   1369.185791   1369.992554   Topic1  -5.5349   1.5185
309       highlight   1361.286377   1362.093262   Topic1  -5.5407   1.5185
366           sheer   1315.615479   1316.422363   Topic1  -5.5748   1.5185
614          colour   1260.435547   1261.242432   Topic1  -5.6177   1.5184
675          subtle   1135.996216   1136.802979   Topic1  -5.7216   1.5184
302         neutral   1035.045898   1035.852783   Topic1  -5.8147   1.5183
694      complexion    920.551697    921.358643   Topic1  -5.9319   1.5182
950            nude    789.669617    790.476440   Topic1  -6.08

In [ ]:
good_reviews = good_lda.get_docs_topics()

In [ ]:
neutral_reviews = neutral_lda.get_docs_topics()

In [ ]:
bad_reviews = bad_lda.get_docs_topics()

In [ ]:
good_reviews.to_pickle('good_reviews.pickle')
bad_reviews.to_pickle('bad_reviews.pickle')
neutral_reviews.to_pickle('neutral_reviews.pickle')

## NLP Preprocessing

In [69]:
docs = pickle.load(open("docs.pickle", "rb"))

d = corpora.Dictionary(docs)

freq = pd.DataFrame(d.dfs.values(), index=d.dfs.keys(), columns=['freq'])
freq.index.name = 'idx'
freq = freq.reset_index()
freq['token'] = freq['idx'].apply(lambda x:d[x])
freq = freq.sort_values(by='freq', ascending=False)

In [75]:
freq[freq['freq']>500]

,idx,freq,token
115,114,1612289,product
8,24,1465876,use
41,44,1461919,love
114,111,1117876,great
14,20,1110963,skin
31,43,1043474,look
5,8,1008814,good
119,117,941979,color
47,45,771895,make
78,70,768706,go


## Validating Model

In [ ]:
# Inputs
docs = random.sample(docs, 100000)

# Create Dictionary
id2word = corpora.Dictionary(docs)
pickle.dump(id2word, open( "id2word.p", "wb" ) )

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, text, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}

nb_words = len(id2word)

# Topics range
min_topics = 6
max_topics = 10
step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
topics_range = [8]
# Alpha parameter
# Added in the loop

# Alpha
alpha = [
#     0.1, 
#     'symmetric',
#     'asymmetric'
]

# Beta parameter
beta = [
    0.1, 
#     200/nb_words
]
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.05), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
    self.corpus
]

corpus_title = [
#     '25% Corpus',
#     '50% Corpus',
#     '75% Corpus',
    '100% Corpus'
]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*(len(alpha)+1)*len(topics_range)))
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            alpha.append(50/k)
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, text=docs,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    pbar.update(1)
    res = pd.DataFrame(model_results)
    res = pd.DataFrame(model_results).sort_values("Coherence", ascending=False)
    res.to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [ ]:
res

## Training Best Model

In [ ]:
best_param = res.iloc[0]
num_topics = best_param['Topics']
alpha = best_param['Alpha']
eta = best_param['Beta']

In [ ]:
# Build LDA model
int_start=time.time()
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       alpha = alpha,
                                       eta = eta,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

In [ ]:
lda_model.save('lda_test.model')

In [ ]:
print(time.time()-start)

## Predictions

In [ ]:
# df = pd.read_pickle('reviews_concat.pkl')

In [ ]:
# df = df.loc['2019']

In [ ]:
# print(len(df))

In [ ]:
# docs = list(tqdm.tqdm(preprocess(df.values.tolist(), stop_words), position=0, leave=True))

In [ ]:
# pickle.dump(docs, open( "docs.p", "wb" ) )

In [ ]:
# docs = pickle.load(open("docs.p", "rb"))
# id2word = pickle.load(open("id2word.p", "rb"))

# # Term Document Frequency
# corpus = [id2word.doc2bow(text) for text in docs]

# output = pd.concat([df.to_frame('description').reset_index(), pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])], axis=1, ignore_index=True)

# output.to_pickle('reviews_w_topics_test.pkl')

## Appendix

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=150) # higher threshold fewer phrases.
# # trigram = gensim.models.Phrases(bigram[data_words], threshold=150)

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# # trigram_mod = gensim.models.phrases.Phraser(trigram)

# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# # Remove Stop Words
# data_words = remove_stopwords(data_words)

# # Do lemmatization keeping only noun, adj, vb, adv
# data_words = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# # Form Bigrams
# data_words = make_bigrams(data_words)

In [ ]:
# # Build LDA model
# lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                        id2word=id2word,
#                                        num_topics=8, 
#                                        random_state=100,
#                                        chunksize=100,
#                                        passes=10,
#                                        per_word_topics=True)

In [ ]:
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [ ]:
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence Score: ', coherence_lda)